In [1]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

In [2]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('C:/projects/face_reco/Deep-Learning-Face-Recognition/haarcascade_frontalface_default.xml')

model = load_model('facefeatures_new_model.h5')


def face_extractor(img):
    faces = face_classifier.detectMultiScale(img, 1.3, 5) 
    faces_cor = []
    face_imgs = np.empty((len(faces), 224, 224, 3))
    
    for i, face in enumerate(faces):  
        if face is ():
            return None, None
        # Crop all faces found
        for (x,y,w,h) in face.reshape((1,4)):
            face_cor = []
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
            x=x-10
            y=y-10
            face_cor.append(x)
            face_cor.append(y)
            face_img = img[y:y+h+50, x:x+w+50]
            try:
                face_img = cv2.resize(face_img, (224, 224))           
                face_imgs[i,:,:,:] = face_img
                faces_cor.append(face_cor)
            except Exception as e:
                return None, None

    return faces_cor, face_imgs

In [3]:
# Doing some Face Recognition with the webcam
video_capture = cv2.VideoCapture(0)
while True:
    valid, frame = video_capture.read()
    faces_cor, faces = face_extractor(frame)
    if faces is not None:
        if faces.shape[0]>0 & valid:
            for i in range(len(faces)):
                face = faces[i]
                face_cor = faces_cor[i]
                face = face/255.0
                face = face.reshape([1,224,224,3])
                pred = model.predict(face)
                #print(pred)
                name="None matching"
                if(pred[0][1]>0.5):
                    name='Piyush'
                    cv2.putText(frame,name, (face_cor[0], face_cor[1]), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                elif (pred[0][0]>0.5):
                    name ='Myra'       
                    cv2.putText(frame,name, (face_cor[0], face_cor[1]), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        else:
            cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2) 
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()